<a href="https://colab.research.google.com/github/shivanigurbani/Analysing-Mental-Health-during-COVID/blob/main/Postcovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                        
**1. Import the Python libraries that we need and then read the CSV train data.**


In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from string import punctuation

#read train csv
tweet_df = pd.read_csv("Post Covid Mini Modified (1).csv", encoding="latin")

To check the data, whether it is loaded or not, I usually check using tweet_df.head().

In [ ]:
tweet_df.head()

,Unnamed: 0,Unnamed: 0.1,message,label,clean_text
0,0,0,@janie_crawford ha...yes ma'am...been branding...,0,thank honor muchim brand maam appreci
1,1,1,@_TASOfficial_ I really miss XYZ since 2016 i ...,1,longest pad anim ever xyz sinc miss post reall...
2,2,2,boys before flowers is AWESOME. specially so y...,0,stori awesom flower befor tri boy gaeul watch ...
3,3,3,RT: #BipolarHotline: National #Suicide Hotline...,1,lockdown pandem suicid mentalhealth hotlin log...
4,4,4,Only 2 days of sophmore year left! thank godd ...,0,day thank year onli godd mehhh left boredd sop...


 2. Clean and Preprocess the Data


In [ ]:
ps = PorterStemmer()
negation_list = ["arent","isnt","dont","doesnt","not","cant","couldnt", "werent",
                 "wont","didnt","never","nothing","nowhere","noone","none"
                "hasnt","hadnt","shouldnt","wouldnt","aint"]
               
def preProcessTweets(tweet):
    tweet = tweet.lower()
    tweet = re.sub('n[^A-Za-z ]t','nt', tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)
    tweet = re.sub('@[^\s]+', '', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = word_tokenize(tweet)
    tweet_list = [];
    negate = False
    for word in tweet:
        word = ps.stem(word)
        if word in negation_list:
            negate = True
        elif negate is True and word in list(punctuation):
            negate = False
            
        if negate and word not in negation_list:
            word = "not_"+word
        else:
            pass
        word = re.sub('[^A-Za-z_ ]+', '', word)
        if len(word) > 2 and word not in stopwords.words('english'):
            tweet_list.append(word)
    tweet_set = set(tweet_list)
    return " ".join(tweet_set)

3. import  nltk                                                                  (nltk to help me preprocess and clean the tweets data.)

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

4. check the new column(clean_text) that contains preprocessed tweets.


In [ ]:
tweet_df["clean_text"] = tweet_df["message"].apply(preProcessTweets)

5. New CSV file, so we don't need to run the preprocess function again.

In [ ]:
tweet_df.to_csv("train_preprocessed.csv")

**Train the Classification Model**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
import pickle

In [ ]:
tweet_df = pd.read_csv("train_preprocessed.csv")

In [ ]:
tweet_df["clean_text"]=tweet_df["clean_text"].astype('U')

In [ ]:
pipeline = Pipeline([
   ( 'bow',CountVectorizer()),
    ('classifier',MultinomialNB()),
])

In [ ]:
from sklearn.model_selection import train_test_split
msg_train,msg_test,label_train,label_test = train_test_split(tweet_df['clean_text'],tweet_df['label'],test_size=0.3)

pipeline.fit(msg_train, label_train)

predictions = pipeline.predict(msg_test)




In [ ]:
print(classification_report(predictions, label_test))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95        86
           1       0.93      0.79      0.86        34

    accuracy                           0.93       120
   macro avg       0.93      0.89      0.90       120
weighted avg       0.93      0.93      0.92       120

